# Automatsko procenjivanje broja učesnika protesta analizom slike

Ovaj projekat se bavi detekcijom i brojanjem ljudi na slikama koristeći HOG, Haar i CNN (density map).
HOG i Haar su istraženi kao opcije ali se ručnim testiranjem ispostavilo da nisu dovoljni za brojanje velike mase ljudi na slikama sa protesta.

Ispostavilo se da je za takve fotografije daleko bolje koristiti CNN sa density mapom koji je fine-tuniran na - ovde konkretno - VisDrone2020-CC lite datasetu.

## 1. HOG detektor

HOG (Histogram of Oriented Gradients) detektuje ljude analizom raspodele ivica i gradijenta na slici. Posebno je efikasan za prepoznavanje pešaka na ulici i ljude čije je celo telo vidljivo.

```bash
cd ../src
python hog_detector.py
```

Skript `hog_detector.py` koristi HOG za prebrojavanje ljudi na tri test slike koje se nalaze u `data/samples`. Rezultati se čuvaju u `data` folderu.

* Na output-u [aerial_hog.jpg](../data/output/aerial_hog.jpg) HOG daje **neprecizan rezultat**, što pokazuje da nije pogodan za scene snimljene iz vazduha.
* Na output-u [closeup_hog.jpg](../data/output/closeup_hog.jpg) HOG **ne daje nikakav rezultat**, što pokazuje da nije pogodan za neprecizan za lica izbliza.
* Na output-u [pedestrians_hog.jpg](../data/output/pedestrians_hog.jpg) HOG daje **dobar rezultat kada je celo telo vidljivo**.

## 2. Haar detektor

Haar Cascade detektor koristi kaskadu jednostavnih vizuelnih karakteristika kako bi brzo odredio da li deo slike sadrži osobu. Posebno je pogodan za **close-up lica** i gornji deo tela.

```bash
cd ../src
python haar_detector.py
```

Skript `haar_detector.py` primenjuje Haar detekciju na iste tri test slike iz `data/samples`. Rezultati se čuvaju u `data` folderu.

* Za sliku [aerial_haar.jpg](../data/output/aerial_haar.jpg) Haar detektor je **neprecizan i detektuje neke pogrešne objekte**, što pokazuje da nije idealan za snimke iz vazduha.
* Za sliku [closeup_haar.jpg](../data/output/closeup_haar.jpg) Haar detektor daje **relativno precizan rezultat za lica izbliza**.
* Za sliku [pedestrians_haar.jpg](../data/output/pedestrians_haar.jpg) detektor je **dobar, ali HOG je precizniji za ljude sa celim telom**.

# CNN Counter (CSRNet)

**CSRNet CNN** model se u ovom projektu koristi za najpreciznije brojanje ljudi na fotografijama slikanim dronom iznad ljudi.
Model predviđa **density mapu** gde je suma svih vrednosti u mapi predviđeni broj ljudi na slici.

## Dataset: `DroneCrowdDataset`
- Čita sekvence iz VisDrone dataset-a.
- Svaka sekvenca ima slike i anotacije (x, y koordinate ljudi).
- Opcionalno se slike dele na tile-ove (`tile_size=(512,512)`).
- Svaka tile slika se pretvara u tensor i normalizuje.
- Density mapa se kreira tako što se svaka anotacija konvertuje u Gaussian kernel.

## Model: `CSRNet`

* Frontend: VGG16 do 30. layer-a (convolutional features)
* Backend: više konvolucionih slojeva sa dilatacijom da bi se sačuvala rezolucija
* Output: 1-channel density mapa iste veličine kao tile

## Trening

* Optimizator: `AdamW`
* Loss: `MSELoss` između predviđene i ground-truth density mape
* Grad scaler za **mixed precision** ako je GPU
* Batch size tipično 8-16 tile-ova
* Epochs: početno 1 za test, kasnije 20 za pravi trening

## Predikcija i vizualizacija

* Model se primenjuje po tile-ovima, density tile se vraća u originalnu veličinu.
* Ukupan broj ljudi se računa kao suma svih vrednosti density mape.

## Napomene

* `test_list.txt` se trenutno ne koristi u treningu, služi samo za eventualno evaluaciju.
* Model je unapred treniran sa VGG16 weights za bolju inicijalizaciju.


## Zaključak

- HOG i Haar su korisni za specifične scenarije, ali nisu dovoljno precizni za proteste.  
- CNN sa density mapama je najprecizniji za sve scenarije i može se fine-tunovati na VisDrone dataset-u.
